In [1]:
import warnings

warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd
import scipy.stats as ss
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
import torch

from tqdm.notebook import tqdm
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from feature_engine.outliers import Winsorizer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif, f_classif
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.kernel_approximation import AdditiveChi2Sampler
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.feature_selection import RFECV, SelectKBest

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.cluster import KMeans, FeatureAgglomeration
from sklearn.utils.class_weight import compute_sample_weight

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [102]:
train = pd.read_csv('./data/train.csv').drop('id', axis=1)
test = pd.read_csv('./data/test.csv').drop('id', axis=1)
sub = pd.read_csv('./data/sample_submission.csv', index_col='id')

In [103]:
train

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,584,590,909972,909977,16,8,5,2274,113,140,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,808,816,728350,728372,433,20,54,44478,70,111,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,39,192,2212076,2212144,11388,705,420,1311391,29,141,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,781,789,3353146,3353173,210,16,29,3202,114,134,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,1540,1560,618457,618502,521,72,67,48231,82,111,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19214,749,757,143210,143219,17,4,4,2193,122,140,...,-0.1429,0.0044,0.2901,0,0,0,1,0,0,0
19215,723,735,2488529,2488541,231,17,26,27135,104,133,...,0.7222,-0.0989,0.5378,0,0,0,0,0,0,1
19216,6,31,1578055,1578129,780,114,98,71112,41,94,...,0.7719,-0.4283,0.9997,1,0,0,0,0,0,0
19217,9,18,1713172,1713184,126,13,26,14808,88,132,...,0.9610,-0.1162,0.3509,0,0,0,0,0,0,1


In [104]:
train.shape, test.shape, sub.shape

((19219, 34), (12814, 27), (12814, 7))

In [105]:
ys = list(train.columns[-7:])

In [106]:
Y = train.loc[:, ys]
X = train.drop(ys, axis=1)

In [107]:
X.isna().sum()

X_Minimum                0
X_Maximum                0
Y_Minimum                0
Y_Maximum                0
Pixels_Areas             0
X_Perimeter              0
Y_Perimeter              0
Sum_of_Luminosity        0
Minimum_of_Luminosity    0
Maximum_of_Luminosity    0
Length_of_Conveyer       0
TypeOfSteel_A300         0
TypeOfSteel_A400         0
Steel_Plate_Thickness    0
Edges_Index              0
Empty_Index              0
Square_Index             0
Outside_X_Index          0
Edges_X_Index            0
Edges_Y_Index            0
Outside_Global_Index     0
LogOfAreas               0
Log_X_Index              0
Log_Y_Index              0
Orientation_Index        0
Luminosity_Index         0
SigmoidOfAreas           0
dtype: int64

In [8]:
# target = np.asarray(Y.apply(lambda x: np.argmax(x), axis=1))
# target

In [9]:
# pd.Series(mutual_info_classif(train.iloc[:, :-7], target), index=train.columns[:-7])

In [10]:
X.corr()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas
X_Minimum,1.000000,0.989767,0.016071,0.014247,-0.464759,-0.451961,-0.426514,-0.430073,0.337748,-0.132955,...,-0.522112,0.112746,0.460421,0.222109,-0.505024,-0.530728,-0.375324,0.276871,-0.068844,-0.376973
X_Maximum,0.989767,1.000000,0.018909,0.016513,-0.391937,-0.378434,-0.356528,-0.363092,0.275571,-0.129669,...,-0.432545,0.095799,0.391743,0.190474,-0.428875,-0.448348,-0.316770,0.237157,-0.079235,-0.319598
Y_Minimum,0.016071,0.018909,1.000000,0.969552,-0.007370,-0.012712,-0.024876,-0.008189,-0.017321,-0.029160,...,0.019685,0.052710,-0.002647,-0.025374,-0.001754,0.015116,-0.023702,-0.044202,-0.023000,-0.031511
Y_Maximum,0.014247,0.016513,0.969552,1.000000,-0.007019,-0.012377,-0.024798,-0.007809,-0.018739,-0.028312,...,0.017271,0.051546,-0.001382,-0.023285,-0.000995,0.015624,-0.022941,-0.042490,-0.022869,-0.030345
Pixels_Areas,-0.464759,-0.391937,-0.007370,-0.007019,1.000000,0.835079,0.834543,0.797843,-0.622867,0.130499,...,0.764810,-0.303335,-0.650704,-0.205579,0.774580,0.749731,0.640033,-0.269013,-0.015081,0.542672
X_Perimeter,-0.451961,-0.378434,-0.012712,-0.012377,0.835079,1.000000,0.912579,0.802072,-0.603891,0.158036,...,0.752470,-0.378672,-0.664781,-0.174615,0.771546,0.750418,0.667441,-0.234969,0.005212,0.561256
Y_Perimeter,-0.426514,-0.356528,-0.024876,-0.024798,0.834543,0.912579,1.000000,0.809171,-0.602826,0.160831,...,0.700036,-0.469092,-0.612157,-0.081225,0.790863,0.705657,0.761117,-0.113968,-0.005650,0.608167
Sum_of_Luminosity,-0.430073,-0.363092,-0.008189,-0.007809,0.797843,0.802072,0.809171,1.000000,-0.580857,0.128569,...,0.703912,-0.315125,-0.608297,-0.177048,0.726483,0.698035,0.615651,-0.234625,-0.013845,0.520282
Minimum_of_Luminosity,0.337748,0.275571,-0.017321,-0.018739,-0.622867,-0.603891,-0.602826,-0.580857,1.000000,0.397265,...,-0.638549,0.234592,0.467841,0.112480,-0.691406,-0.637711,-0.585194,0.179385,0.619174,-0.504692
Maximum_of_Luminosity,-0.132955,-0.129669,-0.029160,-0.028312,0.130499,0.158036,0.160831,0.128569,0.397265,1.000000,...,0.074610,-0.014785,-0.153276,-0.120083,0.033191,0.079426,0.004650,-0.113531,0.853856,0.018501


In [108]:
X['col_1'] = np.sqrt(X['Pixels_Areas'])
X['col_2'] = np.exp(X['LogOfAreas'])
X['col_2_1'] = np.exp(X['Log_X_Index'])
X['col_2_2'] = np.exp(X['Log_Y_Index'])
X['col_3'] = X['X_Perimeter'] / X['Y_Perimeter']
X['col_4'] = (X['X_Maximum'] - X['X_Minimum']) * (X['Y_Maximum'] - X['Y_Minimum'])
X['col_4_1'] = np.abs(X['X_Maximum'] - X['X_Minimum'])
X['col_4_2'] = np.abs(X['Y_Maximum'] - X['Y_Minimum'])
X['col_4_3'] = np.log1p(np.abs(X['X_Maximum'] - X['X_Minimum']))
X['col_4_4'] = np.log1p(np.abs(X['Y_Maximum'] - X['Y_Minimum']))
X[['col_5', 'col_51']] = np.exp(X[['Log_X_Index', 'Log_Y_Index']])

_, bins_1 = pd.qcut(X['LogOfAreas'], q=10, retbins=True, duplicates='drop')
_, bins_2 = pd.qcut(X['Pixels_Areas'], q=10, retbins=True, duplicates='drop')
_, bins_3 = pd.qcut(X['Sum_of_Luminosity'], q=10, retbins=True, duplicates='drop')
_, bins_4 = pd.qcut(X['Steel_Plate_Thickness'], q=10, retbins=True, duplicates='drop')
_, bins_5 = pd.qcut(X['Length_of_Conveyer'], q=10, retbins=True, duplicates='drop')

X['col_6'] = pd.cut(X['LogOfAreas'], bins_1, labels=False, include_lowest=True)
X['col_7'] = pd.cut(X['Pixels_Areas'], bins_2, labels=False, include_lowest=True)
X['col_8'] = pd.cut(X['Sum_of_Luminosity'], bins_3, labels=False, include_lowest=True)
X['col_9'] = pd.cut(X['Steel_Plate_Thickness'], bins_4, labels=False, include_lowest=True)
X['col_10'] = pd.cut(X['Length_of_Conveyer'], bins_5, labels=False, include_lowest=True)

X['col_11'] = X['X_Minimum'] / X['X_Maximum']
X['col_12'] = X['Y_Minimum'] / X['Y_Maximum']
X['col_13'] = X['Sum_of_Luminosity'] / X['Pixels_Areas']
X['col_14'] = X['Minimum_of_Luminosity'] / X['Maximum_of_Luminosity']
X['col_15'] = X['X_Perimeter'] / X['Y_Perimeter']

X['col_16'] = np.cos(np.exp(X['LogOfAreas']))
X['col_17'] = X['Maximum_of_Luminosity'] - X['Minimum_of_Luminosity']
X['col_18'] = np.log1p(X['Sum_of_Luminosity'])
X['col_19'] = np.log1p(X['Length_of_Conveyer'])
X['col_20'] = np.log1p(X['Steel_Plate_Thickness'])

X['col_21'] = (X['Outside_Global_Index'] - X['Edges_Y_Index']) / (X['Outside_X_Index'] * X['Log_X_Index'] ** 2)
X['col_22'] = X['Steel_Plate_Thickness'] / X['Length_of_Conveyer']
X['col_23'] = X['Steel_Plate_Thickness'] / X['Edges_Y_Index'] * X['Outside_X_Index']
X['col_24'] = (X['Log_X_Index'] * X['LogOfAreas']) / X['Edges_Y_Index']
X['col_25'] = X['Pixels_Areas'] / (X['X_Perimeter'] + X['Y_Perimeter'])
X['col_26'] = (X['Pixels_Areas'] / (X['Y_Perimeter'] * X['Edges_Y_Index'])) / (
        X['Sum_of_Luminosity'] - X['Minimum_of_Luminosity'])

In [109]:
# test dataset
test['col_1'] = np.sqrt(test['Pixels_Areas'])
test['col_2'] = np.exp(test['LogOfAreas'])
test['col_2_1'] = np.exp(test['Log_X_Index'])
test['col_2_2'] = np.exp(test['Log_Y_Index'])
test['col_3'] = test['X_Perimeter'] / test['Y_Perimeter']
test['col_4'] = (test['X_Maximum'] - test['X_Minimum']) * (test['Y_Maximum'] - test['Y_Minimum'])
test['col_4_1'] = np.abs(test['X_Maximum'] - test['X_Minimum'])
test['col_4_2'] = np.abs(test['Y_Maximum'] - test['Y_Minimum'])
test['col_4_3'] = np.log1p(np.abs(test['X_Maximum'] - test['X_Minimum']))
test['col_4_4'] = np.log1p(np.abs(test['Y_Maximum'] - test['Y_Minimum']))
test[['col_5', 'col_51']] = np.exp(test[['Log_X_Index', 'Log_Y_Index']])

test['col_6'] = pd.cut(test['LogOfAreas'], bins_1, labels=False, include_lowest=True)
test['col_7'] = pd.cut(test['Pixels_Areas'], bins_2, labels=False, include_lowest=True)
test['col_8'] = pd.cut(test['Sum_of_Luminosity'], bins_3, labels=False, include_lowest=True)
test['col_9'] = pd.cut(test['Steel_Plate_Thickness'], bins_4, labels=False, include_lowest=True)
test['col_10'] = pd.cut(test['Length_of_Conveyer'], bins_5, labels=False, include_lowest=True)

test['col_11'] = test['X_Minimum'] / test['X_Maximum']
test['col_12'] = test['Y_Minimum'] / test['Y_Maximum']
test['col_13'] = test['Sum_of_Luminosity'] / test['Pixels_Areas']
test['col_14'] = test['Minimum_of_Luminosity'] / test['Maximum_of_Luminosity']
test['col_15'] = test['X_Perimeter'] / test['Y_Perimeter']

test['col_16'] = np.cos(np.exp(test['LogOfAreas']))
test['col_17'] = test['Maximum_of_Luminosity'] - test['Minimum_of_Luminosity']
test['col_18'] = np.log1p(test['Sum_of_Luminosity'])
test['col_19'] = np.log1p(test['Length_of_Conveyer'])
test['col_20'] = np.log1p(test['Steel_Plate_Thickness'])

test['col_21'] = (test['Outside_Global_Index'] - test['Edges_Y_Index']) / (
        test['Outside_X_Index'] * test['Log_X_Index'] ** 2)
test['col_22'] = test['Steel_Plate_Thickness'] / test['Length_of_Conveyer']
test['col_23'] = test['Steel_Plate_Thickness'] / test['Edges_Y_Index'] * test['Outside_X_Index']
test['col_24'] = (test['Log_X_Index'] * test['LogOfAreas']) / test['Edges_Y_Index']
test['col_25'] = test['Pixels_Areas'] / (test['X_Perimeter'] + test['Y_Perimeter']) * test['Edges_Y_Index']
test['col_26'] = (test['Pixels_Areas'] / (test['Y_Perimeter'] * test['Edges_Y_Index'])) / (
        test['Sum_of_Luminosity'] - test['Minimum_of_Luminosity'])

In [110]:
test = test.fillna(0)

In [111]:
X.shape, test.shape

((19219, 60), (12814, 60))

In [112]:
pd.Series(ss.kurtosis(X), index=X.columns)

X_Minimum                   -1.327648
X_Maximum                   -1.257175
Y_Minimum                    8.865337
Y_Maximum                    8.839496
Pixels_Areas               181.689607
X_Perimeter                167.364618
Y_Perimeter                  9.289275
Sum_of_Luminosity          121.178595
Minimum_of_Luminosity       -0.003005
Maximum_of_Luminosity        9.781119
Length_of_Conveyer          -1.214170
TypeOfSteel_A300            -1.842475
TypeOfSteel_A400            -1.845782
Steel_Plate_Thickness        5.757569
Edges_Index                 -1.206891
Empty_Index                 -0.174450
Square_Index                -1.154852
Outside_X_Index             20.005039
Edges_X_Index               -0.779575
Edges_Y_Index               -0.561567
Outside_Global_Index        -1.819354
LogOfAreas                  -0.253556
Log_X_Index                 -0.064777
Log_Y_Index                 -0.173621
Orientation_Index           -1.093678
Luminosity_Index             6.899609
SigmoidOfAre

In [113]:
test['col_7'] = test['col_7'].astype(int)

In [17]:
# cap_cols = []
# 
# for c in X.columns:
#     if c not in ['col_6', 'col_7', 'col_8', 'col_9', 'col_10', 'TypeOfSteel_A300', 'TypeOfSteel_A400']:
#         cap_cols.append(c)
# 
# capper = Winsorizer(capping_method='iqr')
# capper.fit(X[cap_cols])

In [18]:
# X.loc[:, cap_cols] = capper.transform(X[cap_cols])
# test.loc[:, cap_cols] = capper.transform(test[cap_cols])

In [19]:
# scalable_cols = []
# 
# for c in X.columns:
#     if c not in ['col_6', 'col_7', 'col_8', 'col_9', 'col_10', 'TypeOfSteel_A300', 'TypeOfSteel_A400']:
#         scalable_cols.append(c)
# 
# scaler = MinMaxScaler()
# scaler.fit(X[scalable_cols])

In [20]:
# X.loc[:, scalable_cols] = scaler.transform(X[scalable_cols])
# test.loc[:, scalable_cols] = scaler.transform(test[scalable_cols])

In [114]:
high_informative = ['Pixels_Areas', 'LogOfAreas', 'Outside_X_Index', 'Sum_of_Luminosity', 'Y_Perimeter', 'X_Perimeter',
                    'SigmoidOfAreas', 'X_Minimum', 'X_Maximum', 'Log_X_Index', 'Edges_Index', 'Steel_Plate_Thickness',
                    'Log_Y_Index', 'Minimum_of_Luminosity', 'Edges_Y_Index', 'Length_of_Conveyer',
                    'Edges_X_Index', ]

In [115]:
lda = make_pipeline(LatentDirichletAllocation(9, n_jobs=-1)).fit(X[high_informative])

X.loc[:, [f'lda_{i}' for i in range(9)]] = lda.transform(X[high_informative])
test.loc[:, [f'lda_{i}' for i in range(9)]] = lda.transform(np.abs(test[high_informative]))

In [116]:
svd = make_pipeline(StandardScaler(),
                    TruncatedSVD(n_components=3)).fit(X[high_informative])

X.loc[:, [f'svd_{i}' for i in range(3)]] = svd.transform(X[high_informative])
test.loc[:, [f'svd_{i}' for i in range(3)]] = svd.transform(test[high_informative])

In [117]:
cluster = make_pipeline(StandardScaler(),
                        KMeans(n_clusters=5)).fit(X[high_informative])

X.loc[:, 'clus'] = cluster.predict(X[high_informative])
test.loc[:, 'clus'] = cluster.predict(test[high_informative])

In [118]:
agg = make_pipeline(StandardScaler(),
                    FeatureAgglomeration(3)).fit(X[high_informative])

X.loc[:, [f'agg_{i}' for i in range(3)]] = agg.transform(X[high_informative])
test.loc[:, [f'agg_{i}' for i in range(3)]] = agg.transform(test[high_informative])

In [119]:
X.shape, test.shape

((19219, 76), (12814, 76))

In [23]:
# pd.Series(mutual_info_classif(X, target), index=X.columns)

## models

In [24]:
cv = MultilabelStratifiedKFold(3, shuffle=True, random_state=48)

In [123]:
est = RandomForestClassifier(max_depth=5, n_jobs=-1, random_state=48)

selector = RFECV(est, min_features_to_select=19, cv=cv, scoring='roc_auc')
selector.fit(X, Y)

selector.n_features_

66

In [124]:
X = selector.transform(X)
test = selector.transform(test)

In [ ]:
def object(trial):
    params = dict(num_iterations=1000,  #trial.suggest_int('n_estimators ', 100, 1500),
                  max_depth=trial.suggest_int('max_depth', 1, 5),
                  # num_leaves=trial.suggest_int('num_leaves ', 3, 15),
                  # bagging_freq=trial.suggest_int('bagging_freq', 32, 256),
                  subsample=trial.suggest_float('subsample ', 1e-3, 1e-1),
                  learning_rate=trial.suggest_float('learning_rate', 1e-4, 1e-2),
                  # gamma=trial.suggest_float('gamma ', 1e-3, 1e-1),
                  feature_fraction=trial.suggest_float('feature_fraction ', 7e-1, 9e-1),
                  colsample_bytree=trial.suggest_float('colsample_bytree', 5e-1, 9e-1),
                  colsample_bylevel=trial.suggest_float('colsample_bylevel ', 5e-1, 9e-1),
                  reg_lambda=trial.suggest_float('reg_lambda ', 1e-2, 1),
                  # lambda_l1=trial.suggest_float('reg_lambda ', 1e-3, 1),
                  )

    estimator = MultiOutputClassifier(LGBMClassifier(**params,
                                                     objective='binary',
                                                     device_type='gpu',
                                                     verbosity=-1,
                                                     # task_type="GPU",
                                                     # cat_features=['col_7', 'col_9', 'col_10',
                                                     #               'TypeOfSteel_A300',
                                                     #               'TypeOfSteel_A400'],
                                                     # silent=True,
                                                     # thread_count=-1
                                                     # n_jobs=-1
                                                     ))
    score = []

    for i, (t_idx, v_idx) in enumerate(cv.split(X, Y)):
        train_x, valid_x = X[t_idx, :], X[v_idx, :]
        train_y, valid_y = Y.loc[t_idx, :], Y.loc[v_idx, :]
        preds = np.zeros((len(valid_y), len(ys)))

        # weights = compute_sample_weight(class_weight='balanced', y=train_y)
        estimator.fit(train_x, train_y)

        pred_vals = estimator.predict_proba(valid_x)

        for i in range(len(ys)):
            preds[:, i] = pred_vals[i][:, 1]

        avg_auc = roc_auc_score(valid_y, preds)
        score.append(avg_auc)

    return np.mean(score)


study = optuna.create_study(direction='maximize')
study.optimize(object, n_trials=30)

In [43]:
study.best_params

{'max_depth': 4,
 'subsample ': 0.03329071650502599,
 'learning_rate': 0.007696872860687134,
 'feature_fraction ': 0.7042302378231268,
 'colsample_bytree': 0.7270310744955015,
 'colsample_bylevel ': 0.8125773066822667,
 'reg_lambda ': 0.8751327285847716}

In [47]:
def test_params(model, x, y):
    scores = []

    for i, (t_idx, v_idx) in enumerate(cv.split(x, y)):
        train_x, valid_x = x[t_idx, :], x[v_idx, :]
        train_y, valid_y = y.loc[t_idx, :], y.loc[v_idx, :]
        preds = np.zeros((len(valid_y), len(ys)))

        # weights = compute_sample_weight(class_weight='balanced', y=train_y)

        estimator = model
        estimator.fit(train_x, train_y)

        pred_vals = estimator.predict_proba(valid_x)

        print(f'fold: {i}')

        for _ in range(len(ys)):
            preds[:, _] = pred_vals[_][:, 1]

        score = roc_auc_score(valid_y, preds)

        print(score)

        scores.append(score)

    return np.mean(scores)

In [126]:
cv = MultilabelStratifiedKFold(n_splits=7, shuffle=True, random_state=48)

model = MultiOutputClassifier(XGBClassifier(n_estimators=1250,
                                            max_depth=5,
                                            subsample=0.4145,
                                            learning_rate=0.0049,
                                            colsample_bytree=0.741,
                                            colsample_bylevel=0.819,
                                            reg_alpha=0.755,
                                            device='cuda:0',
                                            verbosity=0))

test_params(model, X, Y)

fold: 0
0.8890829335146392
fold: 1
0.886287420107052
fold: 2
0.8849303843952063
fold: 3
0.8851378610638203
fold: 4
0.8860683882267136
fold: 5
0.8862083525016927
fold: 6
0.8947708045921342


0.8874980206287513

In [67]:
model = MultiOutputClassifier(LGBMClassifier(num_iterations=500,
                                             max_depth=7,
                                             subsample=0.03329071650502599,
                                             learning_rate=0.01366872860687134,
                                             feature_fraction=0.7042302378231268,
                                             colsample_bytree=0.7270310744955015,
                                             colsample_bylevel=0.8125773066822667,
                                             reg_lambda=0.8751327285847716,
                                             device="gpu",
                                             verbose=-1))

test_params(model, X, Y)

fold: 0
0.8858640195631161
fold: 1
0.8819759597861881
fold: 2
0.8828870833924807
fold: 3
0.8836551114498385
fold: 4
0.8843797538071213
fold: 5
0.88199927072007
fold: 6
0.8900359940351085


0.8843995989648462

In [89]:
model = MultiOutputClassifier(CatBoostClassifier(iterations=39,
                                                 depth=5,
                                                 learning_rate=0.15551739,
                                                 task_type="GPU",
                                                 silent=True))

test_params(model, X, Y)

fold: 0
0.8858840673429729
fold: 1
0.8813748674944472
fold: 2
0.8788335216017941
fold: 3
0.8786451460461441
fold: 4
0.8833795037075305
fold: 5
0.8836285910142415
fold: 6
0.8884995822289395


0.8828921827765813

In [131]:
class SteelDataset(Dataset):

    def __init__(self, x, y):
        self.x = x.reset_index(drop=True)
        self.y = y.reset_index(drop=True)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        xs = self.x.loc[idx, :]
        ys = self.y.loc[idx, :]

        return torch.tensor(xs.values, dtype=torch.float32), torch.tensor(ys.values, dtype=torch.float32)


class SteelModel(nn.Module):

    def __init__(self, in_channels, out_channels):
        super(SteelModel, self).__init__()

        self.h_channels = 64
        self.lstm_channels = 128

        self.tail = nn.Sequential(nn.Linear(in_channels, self.h_channels),
                                  nn.LeakyReLU(),
                                  nn.BatchNorm1d(self.h_channels),
                                  nn.Linear(self.h_channels, self.h_channels),
                                  nn.LeakyReLU(),
                                  nn.BatchNorm1d(self.h_channels),
                                  nn.Dropout(0.3))
        self.lstm = nn.LSTM(self.h_channels, self.lstm_channels, batch_first=True, bidirectional=True)
        self.head = nn.Sequential(nn.Linear(2 * self.lstm_channels, self.h_channels),
                                  nn.LeakyReLU(),
                                  nn.BatchNorm1d(self.h_channels),
                                  nn.Dropout(0.2),
                                  nn.Linear(self.h_channels, out_channels),
                                  nn.Sigmoid())

    def forward(self, x):
        x = self.tail(x)
        x, _ = self.lstm(x)
        x = self.head(x)

        return x

In [102]:
def move_to(obj, device):
    """
    move data from tensor from back and forth

    :param obj: the python object to move to a device, or to move its
                contents to a device
    :param device: the compute device to move objects to

    :return:
    """
    if isinstance(obj, list):
        return [move_to(x, device) for x in obj]

    elif isinstance(obj, tuple):
        return tuple(move_to(list(obj), device))

    elif isinstance(obj, set):
        return set(move_to(list(obj), device))

    elif isinstance(obj, dict):
        to_ret = dict()

        for key, value in obj.items():
            to_ret[move_to(key, device)] = move_to(value, device)

        return to_ret

    elif hasattr(obj, "to"):
        return obj.to(device)

    else:
        return obj

In [135]:
epochs = 50
test_preds = []

for r in range(3):
    print(f'repeat: {r}')

    cv = MultilabelStratifiedKFold(n_splits=3, shuffle=True)

    for i, (t_idx, v_idx) in enumerate(cv.split(X, Y)):
        train_x, valid_x = X.loc[t_idx, :], X.loc[v_idx, :]
        train_y, valid_y = Y.loc[t_idx, :], Y.loc[v_idx, :]
        pred_test = np.zeros((len(test), len(ys)))
        best_loss = 1e5

        train_dataset = SteelDataset(train_x, train_y)
        train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, drop_last=True)
        valid_dataset = SteelDataset(valid_x, valid_y)
        valid_loader = DataLoader(valid_dataset, batch_size=512, shuffle=True, drop_last=True)

        loss_func = nn.BCELoss()
        model = SteelModel(57, 7).cuda()
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
        lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.33, patience=3, cooldown=3,
                                                                  verbose=True)

        print(f'fold: {i}')

        for epoch in tqdm(range(epochs)):
            runing_t_loss = []
            runing_t_auc = []
            runing_v_loss = []
            runing_v_auc = []

            print(f'epoch: {epoch}')

            model.train()

            for t_x, t_y in train_loader:
                t_x = move_to(t_x, device='cuda')
                t_y = move_to(t_y, device='cuda')

                y_hat = model(t_x)
                loss = loss_func(y_hat, t_y)
                runing_t_loss.append(loss.detach().cpu().numpy())

                if model.training:
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()

                runing_t_auc.append(
                    np.mean(
                        [roc_auc_score(t_y.detach().cpu().numpy()[:, i], y_hat.detach().cpu().numpy()[:, i]) for i in
                         range(len(ys))]))

            print(f'train loss: {np.mean(runing_t_loss)} train AUC: {np.mean(runing_t_auc)}')

            model.eval()

            for v_x, v_y in valid_loader:
                v_x = move_to(v_x, device='cuda')
                v_y = move_to(v_y, device='cuda')

                with torch.no_grad():
                    val_hat = model(v_x)
                    val_loss = loss_func(val_hat, v_y)
                    runing_v_loss.append(val_loss.detach().cpu().numpy())

                runing_v_auc.append(
                    np.mean(
                        [roc_auc_score(v_y.detach().cpu().numpy()[:, i], val_hat.detach().cpu().numpy()[:, i]) for i in
                         range(len(ys))]))

            lr_scheduler.step(np.mean(runing_v_loss))

            print(f'val loss: {np.mean(runing_v_loss)} val AUC: {np.mean(runing_v_auc)}')

            if np.mean(runing_v_loss) < best_loss:
                best_loss = np.mean(runing_v_loss)

                torch.save({'model_state_dict': model.state_dict()}, f'./ann_cps/checkpoint_{r}_{i}.pth')

                print('best checkpoint updated')

        with torch.no_grad():
            test_pred = model(torch.tensor(test.values, dtype=torch.float32, device='cuda'))

        test_preds.append(test_pred.detach().cpu().numpy())

repeat: 0
fold: 0


  0%|          | 0/50 [00:00<?, ?it/s]

epoch: 0
train loss: 0.633056640625 train AUC: 0.721029335821105
val loss: 0.6180182099342346 val AUC: 0.7746270599767734
best checkpoint updated
epoch: 1
train loss: 0.5799828171730042 train AUC: 0.7818356681695446
val loss: 0.5525301098823547 val AUC: 0.8255102821837389
best checkpoint updated
epoch: 2
train loss: 0.5335808396339417 train AUC: 0.7943458096179207
val loss: 0.4868198335170746 val AUC: 0.8306506804113093
best checkpoint updated
epoch: 3
train loss: 0.47123634815216064 train AUC: 0.8015069480809615
val loss: 0.4268195629119873 val AUC: 0.8453807477296147
best checkpoint updated
epoch: 4
train loss: 0.3995712697505951 train AUC: 0.8002645363193162
val loss: 0.35513123869895935 val AUC: 0.8442518634724704
best checkpoint updated
epoch: 5
train loss: 0.3403877913951874 train AUC: 0.8055000703585656
val loss: 0.3086388409137726 val AUC: 0.8532436798958742
best checkpoint updated
epoch: 6
train loss: 0.3009088635444641 train AUC: 0.819546818454243
val loss: 0.2754227221012115

  0%|          | 0/50 [00:00<?, ?it/s]

epoch: 0
train loss: 0.6531344652175903 train AUC: 0.7219315346264972
val loss: 0.6454449892044067 val AUC: 0.7896642840709488
best checkpoint updated
epoch: 1
train loss: 0.5966870784759521 train AUC: 0.7776177991787112
val loss: 0.5703415870666504 val AUC: 0.8193157632895808
best checkpoint updated
epoch: 2
train loss: 0.5473779439926147 train AUC: 0.784618053106325
val loss: 0.5061435103416443 val AUC: 0.8302752502607468
best checkpoint updated
epoch: 3
train loss: 0.4811159074306488 train AUC: 0.7901166233967296
val loss: 0.4411843717098236 val AUC: 0.8353945569498467
best checkpoint updated
epoch: 4
train loss: 0.4052644371986389 train AUC: 0.7911779852943419
val loss: 0.3728468716144562 val AUC: 0.8497075028355855
best checkpoint updated
epoch: 5
train loss: 0.3510490655899048 train AUC: 0.7989317319391541
val loss: 0.3231152594089508 val AUC: 0.8535714502106312
best checkpoint updated
epoch: 6
train loss: 0.30675673484802246 train AUC: 0.8095254705539254
val loss: 0.286006122827

...

epoch: 0
train loss: 0.666463315486908 train AUC: 0.7214961629399411
val loss: 0.6504760980606079 val AUC: 0.7912350068394899
best checkpoint updated
epoch: 1
train loss: 0.6066562533378601 train AUC: 0.7843592756375137
val loss: 0.5738229751586914 val AUC: 0.8210314945591753
best checkpoint updated
epoch: 2
train loss: 0.5593487024307251 train AUC: 0.7932167163566552
val loss: 0.5163301229476929 val AUC: 0.8304913407686646
best checkpoint updated
epoch: 3
train loss: 0.49409905076026917 train AUC: 0.7977944493095299
val loss: 0.44765856862068176 val AUC: 0.8370527020944426
best checkpoint updated
epoch: 4
train loss: 0.4183894395828247 train AUC: 0.8017316698615982
val loss: 0.37071502208709717 val AUC: 0.835394085886461
best checkpoint updated
epoch: 5
train loss: 0.3539106845855713 train AUC: 0.805687385106525
val loss: 0.31628501415252686 val AUC: 0.8479575051855185
best checkpoint updated
epoch: 6
train loss: 0.3089357018470764 train AUC: 0.8131550972674692
val loss: 0.28231051564

  0%|          | 0/50 [00:00<?, ?it/s]

epoch: 0
train loss: 0.6474987864494324 train AUC: 0.7345427769853816
val loss: 0.6387002468109131 val AUC: 0.7733859628591141
best checkpoint updated
epoch: 1
train loss: 0.5932794809341431 train AUC: 0.7830088147723964
val loss: 0.5572831034660339 val AUC: 0.8274675409541726
best checkpoint updated
epoch: 2
train loss: 0.5430582165718079 train AUC: 0.798857294490543
val loss: 0.4947735369205475 val AUC: 0.8363252108307827
best checkpoint updated
epoch: 3
train loss: 0.47787773609161377 train AUC: 0.7953283713184188
val loss: 0.4175068140029907 val AUC: 0.848389006685708
best checkpoint updated
epoch: 4
train loss: 0.4038950204849243 train AUC: 0.8052066722954936
val loss: 0.3768129050731659 val AUC: 0.8451130245558341
best checkpoint updated
epoch: 5
train loss: 0.34146472811698914 train AUC: 0.810558533423802
val loss: 0.32505980134010315 val AUC: 0.8502037412833433
best checkpoint updated
epoch: 6
train loss: 0.3042070269584656 train AUC: 0.8155587744395139
val loss: 0.312250882387

  0%|          | 0/50 [00:00<?, ?it/s]

epoch: 0
train loss: 0.6539062261581421 train AUC: 0.7252673108486125
val loss: 0.6417168974876404 val AUC: 0.7823715845153169
best checkpoint updated
epoch: 1
train loss: 0.5954564809799194 train AUC: 0.7839793965087631
val loss: 0.5548148155212402 val AUC: 0.8118465886202649
best checkpoint updated
epoch: 2
train loss: 0.548616886138916 train AUC: 0.790397561810151
val loss: 0.49916598200798035 val AUC: 0.8348581446008012
best checkpoint updated
epoch: 3
train loss: 0.48421671986579895 train AUC: 0.7994962309941497
val loss: 0.4385284185409546 val AUC: 0.8382663771139137
best checkpoint updated
epoch: 4
train loss: 0.41081658005714417 train AUC: 0.7984596780013866
val loss: 0.3641849756240845 val AUC: 0.850181671849141
best checkpoint updated
epoch: 5
train loss: 0.34831303358078003 train AUC: 0.7986093184644798
val loss: 0.3102281391620636 val AUC: 0.8517128622054747
best checkpoint updated
epoch: 6
train loss: 0.3043479323387146 train AUC: 0.8106704317034819
val loss: 0.28241237998

In [129]:
estimators = dict(lgb=LGBMClassifier(num_iterations=500,
                                     max_depth=7,
                                     subsample=0.03329071650502599,
                                     learning_rate=0.01366872860687134,
                                     feature_fraction=0.7042302378231268,
                                     colsample_bytree=0.7270310744955015,
                                     colsample_bylevel=0.8125773066822667,
                                     reg_lambda=0.8751327285847716,
                                     device="gpu",
                                     verbose=-1),
                  xgb=XGBClassifier(n_estimators=1250,
                                    max_depth=5,
                                    subsample=0.4145,
                                    learning_rate=0.0049,
                                    colsample_bytree=0.741,
                                    colsample_bylevel=0.819,
                                    reg_alpha=0.755,
                                    device='cuda:0',
                                    verbosity=0),
                  cgb=CatBoostClassifier(iterations=39,
                                         depth=5,
                                         learning_rate=0.15551739,
                                         task_type="GPU",
                                         silent=True)
                  )

In [130]:
def run(estimators_, x, y):
    preds = {}

    for k, est in estimators_.items():
        m_preds = []
        print(f'estimator: {k}')

        for r in range(5):
            print(f'repeat: {r}')

            cv = MultilabelStratifiedKFold(n_splits=7, shuffle=True)

            for i, (t_idx, v_idx) in enumerate(cv.split(x, y)):
                train_x, valid_x = x[t_idx, :], x[v_idx, :]
                train_y, valid_y = y.loc[t_idx, :], y.loc[v_idx, :]
                pred_test = np.zeros((len(test), len(ys)))

                estimator = make_pipeline(MultiOutputClassifier(est))
                estimator.fit(train_x, train_y)

                pred_vals = estimator.predict_proba(valid_x)

                print(f'fold: {i}')
                print(np.mean([roc_auc_score(valid_y.iloc[:, i], pred_vals[i][:, 1]) for i in range(len(ys))]))

                for j in range(len(ys)):
                    pred_test[:, j] = estimator.predict_proba(test)[j][:, 1]

                m_preds.append(pred_test)

        preds[k] = m_preds

    return preds

In [131]:
preds = run(estimators, X, Y)

estimator: lgb
repeat: 0
fold: 0
0.8850161298551269
fold: 1
0.8887909836908212
fold: 2
0.8874720632415666
fold: 3
0.8814420355848911
fold: 4
0.8843736463138594
fold: 5
0.8888730407538519
fold: 6
0.8829150828762089
repeat: 1
fold: 0
0.8929253260937143
fold: 1
0.8809332086061294
fold: 2
0.8832010873857955
fold: 3
0.8944662143428027
fold: 4
0.8823926627955523
fold: 5
0.8841169241306048
fold: 6
0.8849855433016051
repeat: 2
fold: 0
0.8810249067355429
fold: 1
0.8812835246223854
fold: 2
0.8834453453916432
fold: 3
0.8910583499858805
fold: 4
0.8843399250140769
fold: 5
0.8936923833094216
fold: 6
0.8912170910928066
repeat: 3
fold: 0
0.8833895374550019
fold: 1
0.894965869717217
fold: 2
0.879010495626514
fold: 3
0.8822246577108181
fold: 4
0.8866189231115683
fold: 5
0.8869232859453698
fold: 6
0.886536623541838
repeat: 4
fold: 0
0.8862111186459224
fold: 1
0.8940196230139846
fold: 2
0.8828918531662892
fold: 3
0.8852402729775601
fold: 4
0.8914164430188798
fold: 5
0.8750229295968743
fold: 6
0.8853314336

In [143]:
ann = np.mean(np.asarray(test_preds), axis=0)

In [133]:
submission = (np.mean(np.asarray(preds['lgb']), axis=0) +
              # 0.2 * ann +
              np.mean(np.asarray(preds['xgb']), axis=0) +
              np.mean(np.asarray(preds['cgb']), axis=0))/ 3
submission = pd.DataFrame(submission, columns=sub.columns).set_index(sub.index)
submission.to_csv('submission_2.csv')

## sudo labeling

In [67]:
mask = np.zeros_like(submission.values)
mask[np.arange(len(submission)), submission.values.argmax(1)] = 1

test_sudo = pd.DataFrame(mask, columns=submission.columns).astype(int)
test_sudo

,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...
12809,0,0,0,0,0,0,1
12810,0,0,0,0,0,0,1
12811,0,0,1,0,0,0,0
12812,1,0,0,0,0,0,0


In [68]:
new_train = pd.concat((X, test), axis=0)
new_y = pd.concat((Y, test_sudo), axis=0)

In [69]:
new_train.reset_index(drop=True, inplace=True), new_y.reset_index(drop=True, inplace=True)

(None, None)

In [71]:
cv = MultilabelStratifiedKFold(n_splits=7, shuffle=True)

## adjested parameters

In [73]:
estimators = dict(lgb=LGBMClassifier(num_iterations=1000,
                                     max_depth=3,
                                     learning_rate=0.0098988815333333,
                                     feature_fraction=0.8578138132619784,
                                     colsample_bytree=0.8768880865721203,
                                     reg_lambda=0.7173767456929144,
                                     device="gpu",
                                     verbose=-1),
                  xgb=XGBClassifier(n_estimators=1000,
                                    max_depth=6,
                                    subsample=0.5342564004636723,
                                    learning_rate=0.0064835939090773,
                                    colsample_bytree=0.8195724454605662,
                                    reg_lambda=0.09829546268908843,
                                    device='cuda:0',
                                    verbosity=0),
                  cgb=CatBoostClassifier(iterations=110,
                                         depth=6,
                                         learning_rate=0.06201739,
                                         task_type="GPU",
                                         cat_features=['col_7', 'col_9', 'col_10',
                                                       'TypeOfSteel_A300',
                                                       'TypeOfSteel_A400'],
                                         silent=True)
                  )

In [74]:
preds_sudo = run(estimators, new_train, new_y)

estimator: lgb
repeat: 0
fold: 0
0.9235298340826964
fold: 1
0.920031843730221
fold: 2
0.925597484617836
fold: 3
0.9254829579036551
fold: 4
0.919936498938264
fold: 5
0.9173495225279107
fold: 6
0.9177276349419298
repeat: 1
fold: 0
0.920152108402571
fold: 1
0.914422992867854
fold: 2
0.9223185143707201
fold: 3
0.9210876881754031
fold: 4
0.9234152922210682
fold: 5
0.9212804576067073
fold: 6
0.9278080851442703
repeat: 2
fold: 0
0.9199293834871664
fold: 1
0.9191341820742318
fold: 2
0.9193754152224337
fold: 3
0.9175908421465644
fold: 4
0.926162562365645
fold: 5
0.9225567305225415
fold: 6
0.9239758035880096
repeat: 3
fold: 0
0.9195858608549977
fold: 1
0.9162450151379825
fold: 2
0.9256171048837388
fold: 3
0.9283986278802904
fold: 4
0.9210490869989457
fold: 5
0.9178742574107288
fold: 6
0.9230452214652564
repeat: 4
fold: 0
0.9189878545360827
fold: 1
0.9260340115851077
fold: 2
0.9214153499648114
fold: 3
0.919320059251012
fold: 4
0.9197256080084049
fold: 5
0.9188078459221057
fold: 6
0.92114388589180

In [76]:
submission_sudo = (0.1 * np.mean(np.asarray(preds_sudo['lgb']), axis=0) +
                   0.6 * np.mean(np.asarray(preds_sudo['xgb']), axis=0) +
                   0.3 * np.mean(np.asarray(preds_sudo['cgb']), axis=0))
submission_sudo = pd.DataFrame(submission_sudo, columns=sub.columns).set_index(sub.index)
submission_sudo.to_csv('submission_sudo.csv')
submission_sudo

,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
id,,,,,,,
19219,0.600816,0.002274,0.002021,0.000160,0.007662,0.097604,0.467252
19220,0.186851,0.009450,0.007282,0.000260,0.105812,0.148495,0.416563
19221,0.001967,0.018239,0.019993,0.000506,0.005504,0.231412,0.615824
19222,0.064271,0.001191,0.000393,0.001418,0.005827,0.402230,0.455761
19223,0.002356,0.001193,0.000817,0.001439,0.002905,0.729605,0.302489
...,...,...,...,...,...,...,...
32028,0.030502,0.059271,0.001424,0.000174,0.014847,0.121276,0.573928
32029,0.079953,0.002290,0.012762,0.003498,0.077682,0.127815,0.621720
32030,0.000470,0.000617,0.949288,0.000181,0.000325,0.001405,0.048698
